In [2]:
from sympy import *
from sympy.polys.matrices import DomainMatrix
from sympy.polys.factortools import dup_factor_list
from sympy.polys.agca.extensions import FiniteExtension
from functools import reduce


def basic_cols(A):
    """Find basic columns of $A$ that spans $R(A)$ by gaussian elimination"""
    _, pivots = A.rref()
    basic_cols = [A[:, p] for p in pivots]
    return reduce(DomainMatrix.hstack, basic_cols)


def basic_cols_independent(A, B):
    """Find basic columns of $B$ that are linearly independent of the columns of $A$"""
    temp = A.hstack(B)
    _, pivots = temp.rref()
    basic_cols = [temp[:, p] for p in pivots if p >= A.shape[1]]
    if basic_cols:
        return reduce(DomainMatrix.hstack, basic_cols)
    return DomainMatrix.zeros((A.shape[0], 0), A.domain)


def particular_solution(A, b):
    """Find a particular solution $x$ of $A x = b$ by gaussian elimination if $b \in R(A)$"""
    aug = A.hstack(b)
    rref, pivots = aug.rref()
    sol = {}
    for i, j in enumerate(pivots):
        if j >= A.shape[1]:
            continue
        sol[j] = {0: rref[i, -1].element}
    return DomainMatrix(sol, (A.shape[1], 1), b.domain)


def intersection_basis(A, B):
    """Compute the basis of $N(A) \cap R(B)$"""
    X = basic_cols(B)
    V = (A * X).nullspace().transpose()
    return X * V
    

def jordan_chain(L, x, i):
    """Compute the jordan chain $L^{i-1} x, \cdots, L x, x$"""
    jordan_chain = [x]
    for k in range(i):
        jordan_chain.append(L * jordan_chain[-1])
    jordan_chain.reverse()
    jordan_chain = reduce(DomainMatrix.hstack, jordan_chain)
    return jordan_chain


def rank(A):
    """Compute the matrix rank"""
    _, pivots = A.rref()
    return len(pivots)


def jordan_form(A):
    """Compute jordan structure of DomainMatrix"""
    charpoly = A.charpoly()
    n, _ = A.shape
    domain = A.domain
    _, factors = dup_factor_list(charpoly, domain)
    l = Symbol('lambda')
    
    algebraic_jordan_structure = []
    for base, exp in factors:
        mod = Poly.from_list(base, l, domain=domain)
        extension = FiniteExtension(mod)
        
        # Set $L = A - \lambda I$
        L = A.convert_to(extension) - extension(l) * DomainMatrix.eye(n, extension)
        
        # Find nilpotent index
        for k in range(n):
            if rank(L**k) == rank(L**(k + 1)):
                break
        
        # Compute the list $S_0, \cdots, S_{k-1}$ where
        # $S_{k-1} = R(L^{k-1}) \cap N(L)$
        # $S_{k-1} \cup S_{k-2} = R(L^{k-2}) \cap N(L)$
        # $\vdots$
        # $S_{k-1} \cup S_{k-2} \cup \cdots \cup S_{1} = R(L) \cap N(L)$
        # $S_{k-1} \cup S_{k-2} \cup \cdots \cup S_{1} \cup S_{0} = N(L)$
        S_list = [intersection_basis(L, L**(k - 1))]
        for i in range(1, k):
            BV = intersection_basis(L, L**(k - i - 1))
            Y = reduce(DomainMatrix.hstack, S_list)
            S = basic_cols_independent(Y, BV)
            S_list.append(S)
        S_list.reverse()
        
        # Build jordan chains with respect to each generalized eigenvectors
        # By solving $L^{i} x = b$ for each $b \in S_{i}$
        # And build jordan chain $L^{i-1} x, \cdots, L x, x$
        jordan_chains = []
        for i, S in reversed(tuple(enumerate(S_list))):
            for j in range(S.shape[1]):
                x = particular_solution(L**i, S[:, j])
                jordan_chains.append(jordan_chain(L, x, i))

        algebraic_jordan_structure.append((extension, mod, jordan_chains))
    return algebraic_jordan_structure


def jordan_form_to_sympy(algebraic_jordan_structure):
    """Convert the result of the jordan structure to sympy matrix"""
    J = []
    P = []
    for extension, mod, jordan_chains in algebraic_jordan_structure:
        l = mod.gens[0]
        degree = mod.degree()
        
        eigenvals = roots(mod, l)
        if len(eigenvals) != degree:
            eigenvals = [CRootOf(mod, l, idx) for idx in range(degree)]

        jordan_chains = [jordan_chain.to_Matrix() for jordan_chain in jordan_chains]
        for eigenval in eigenvals:
            for jordan_chain in jordan_chains:
                jordan_chain = jordan_chain.xreplace({l: eigenval})
                P.append(jordan_chain)
                jordan_block = Matrix.jordan_block(jordan_chain.shape[1], eigenval)
                J.append(jordan_block)
                
    P = Matrix.hstack(*P)
    J = BlockDiagMatrix(*J).as_explicit()
    return P, J


def DOM_jordan_form(A):
    A = DomainMatrix.from_Matrix(A)
    A = A.to_field()
    algebraic_jordan_structure = jordan_form(A)
    P, J = jordan_form_to_sympy(algebraic_jordan_structure)
    return P, J

In [4]:
A = Matrix([
    [-4, -5, -3, 1, -2, 0, 1, -2],
    [4, 7, 3, -1, 3, 0, -1, 2],
    [0, -1, 0, 0, 0, 0, 0, 0],
    [-1, 1, 2, -4, 2, 0, -3, 1],
    [-8, -14, -5, 1, -6, 0, 1, -4],
    [4, 7, 4, -3, 3, -1, -3, 4],
    [2, -2, -2, 5, -3, 0, 4, -1],
    [6, 7, 3, 0, 2, 0, 0, 3]])

P, J = DOM_jordan_form(A)
print("Diagonal matrix:")
display(P)
print("Nilpotent matrix:")
display(J)
display(P*J*P.inv())

Diagonal matrix:


Matrix([
[ 1/11, -3/11,  1, -1, -1,  0, -1, 0],
[-4/11,  1/11,  0,  1,  0,  0,  0, 0],
[ 4/11, -1/11, -1,  0,  2,  0,  0, 0],
[-4/11,  1/11, -1,  1,  1,  1,  1, 0],
[ 5/11, -4/11,  0, -1,  0,  0,  1, 0],
[-7/11, 10/11, -1,  1,  0,  0,  0, 1],
[    1,     0,  0, -1,  0, -1,  0, 0],
[    0,     1, -1,  0,  0,  0,  1, 0]])

Nilpotent matrix:


Matrix([
[1, 0, 0, 0, 0,  0,  0,  0],
[0, 1, 0, 0, 0,  0,  0,  0],
[0, 0, 0, 1, 0,  0,  0,  0],
[0, 0, 0, 0, 1,  0,  0,  0],
[0, 0, 0, 0, 0,  0,  0,  0],
[0, 0, 0, 0, 0, -1,  1,  0],
[0, 0, 0, 0, 0,  0, -1,  0],
[0, 0, 0, 0, 0,  0,  0, -1]])

Matrix([
[-4,  -5, -3,  1, -2,  0,  1, -2],
[ 4,   7,  3, -1,  3,  0, -1,  2],
[ 0,  -1,  0,  0,  0,  0,  0,  0],
[-1,   1,  2, -4,  2,  0, -3,  1],
[-8, -14, -5,  1, -6,  0,  1, -4],
[ 4,   7,  4, -3,  3, -1, -3,  4],
[ 2,  -2, -2,  5, -3,  0,  4, -1],
[ 6,   7,  3,  0,  2,  0,  0,  3]])